In [1]:
import pandas as pd

In [2]:
REGISTRO_KEYS = ['Cotización', 'Fecha', 'Nombre', 'Correo', 'Teléfono', 'Institución', 'Interno',
                  'Responsable', 'Muestra', 'Equipo', 'Estado', 'Pago', 'Referencia', 'Tipo de Pago', 'Valor']
CLIENTES_KEYS = ['Nombre', 'Correo', 'Teléfono', 'Institución', 'Documento',
                'Dirección', 'Ciudad', 'Interno', 'Responsable', 'Proyecto', 'Código', 'Tipo de Pago']

In [3]:
afm = pd.read_excel("AFM.xlsx").fillna("").astype(str)
con = pd.read_excel("Confocal.xlsx").fillna("").astype(str)
sem = pd.read_excel("SEM.xlsx").fillna("").astype(str)

clientes = pd.read_excel("Clientes.xlsx").fillna("").astype(str)

In [4]:
def resume_category(df, key):
    temp = df.groupby(key)
    text = "\n".join(["%s: %d"%(a, len(b)) for (a, b) in temp])
    return text

# Title fields

In [5]:
for key in CLIENTES_KEYS:
    if not key in list(clientes.keys()):
        clientes[key] = pd.Series([""]*len(clientes), index = clientes.index)
    series = clientes[key]
    if key != "Correo": series = series.str.title()
    else: series = series.str.lower()
    series = series.str.replace("\n", "").str.replace("\r", "").str.strip()
    clientes[key] = series

# Drop Duplicates

In [6]:
clientes = clientes.drop_duplicates("Correo", 'first')
clientes = clientes.sort_values("Nombre")

# Valid user

In [7]:
valid = (clientes["Interno"] == "Interno") | (clientes["Interno"] == "Externo")
clientes = clientes[valid]

# Cities

In [8]:
def replace(df, key, prev, new):
    df[key][df[key] == prev] = new

In [9]:
replace(clientes, "Ciudad", "Bogota D.C", "Bogotá D.C.")
replace(clientes, "Ciudad", "Botoga", "Bogotá D.C.")
replace(clientes, "Ciudad", "Bogota", "Bogotá D.C.")
replace(clientes, "Ciudad", "Bogotá", "Bogotá D.C.")
replace(clientes, "Ciudad", "Bogotá D.C", "Bogotá D.C.")

replace(clientes, "Ciudad", "Bogotá D.C.", "Bogotá")

replace(clientes, "Ciudad", "Cartagena-Bolívar", "Cartagena")
replace(clientes, "Ciudad", "Medellin", "Medellín")
replace(clientes, "Ciudad", "Santa Marta - Colombia", "Santa Marta")

In [10]:
values = resume_category(clientes, "Ciudad")
print(values)

: 6
Ambalá: 1
Armenia: 1
Barranquilla: 3
Bogotá: 403
Bucaramanga: 1
Cajicá: 1
Cali: 2
Cartagena: 2
Cundinamarca: 1
Envigado: 1
Facatativá: 1
Medellín: 2
Neiva: 2
Santa Marta: 1
Tunja: 7
Valle Del Cauca: 1
Valledupar: 2


In [11]:
clientes = clientes.reset_index(drop = True)
clientes = clientes[CLIENTES_KEYS]
clientes.to_excel("Clientes.xlsx", index = False)

In [12]:
resume = resume_category(clientes, "Institución")
print(resume)

: 10
-: 1
Agp Glass: 1
Alkhorayef Zona Franca: 1
Andimallas & Andimetales S.A.: 1
Antropología: 1
Arte Y Humanidades: 1
Bd Medical: 1
Belcorp: 1
Biología: 1
Biomed, Uniandes: 1
Biommar: 4
C. Biomedicas: 1
Cga: 1
Ciencias Biologícas: 1
Ciencias Biológicas: 14
Cieo: 2
Cigen Uniandes: 1
Cimic: 4
Cirujano Y Oftanmologo: 1
Coc: 1
Colegio Odontológico: 2
Corhuila: 1
Corpogen: 1
Corpoica, C.I. Tibaitatá: 1
Corporación Universitaria Minuto De Dios: 1
Cromasoft Ltda: 1
Departamento De Física: 1
Departamento De Química: 1
Depto Química. Uniandes: 1
Diseño: 3
Ecci: 1
Externo: 10
Facultad De Ingeniería: 1
Facultad De Odontología De La Universidad Ces: 1
Ferrotérmicos: 1
Fisica: 2
Fundación Cardioinfantil - Instituto De Cardiología: 1
Fundación Universitaria Del Área Andina: 1
Física: 11
General Motors Colmotores S.A: 1
Genética Humana.: 1
Geociencias: 10
Geocol: 1
Geoimec S.A.S: 1
Gmas S.A.S.: 2
Gmas Sas: 1
Haerentia Sas: 1
Hospital De San Jose: 1
Imec: 5
Ing Biomedica: 1
Ing Biomédica: 2
Ing Elec

# Merge equipos

In [13]:
afm["Cotización"] = ["A18-%04d"%int(item) for item in afm["Cotización"].values]
con["Cotización"] = ["C18-%04d"%int(item) for item in con["Cotización"].values]
sem["Cotización"] = ["M18-%04d"%int(item) for item in sem["Cotización"].values]

In [14]:
afm["Equipo"] = "AFM"
con["Equipo"] = "Confocal"
sem["Equipo"] = "MBE"

In [15]:
merged = afm.append(con).append(sem)

In [16]:
merged = merged.drop_duplicates("Cotización", "first")

In [17]:
for key in REGISTRO_KEYS:
    if not key in list(merged.keys()):
        merged[key] = pd.Series([""]*len(merged), index = merged.index)
    series = merged[key]
    if key == "Equipo": pass
    elif key != "Correo": series = series.str.title()
    else: series = series.str.lower()
    series = series.str.replace("\n", "").str.replace("\r", "").str.strip()
    merged[key] = series

In [18]:
merged = merged.sort_values("Cotización", ascending = False)
merged = merged[REGISTRO_KEYS]

merged.to_excel("Registro.xlsx", index = False)
